In [1]:
import pandas as pd
import urllib.request
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from haversine import haversine, Unit
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import pytz
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score

In [2]:
# import necessary libraries
import pandas as pd
import os
import glob
  
  
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "temp/*_hard_voting.csv"))
  
master_df = pd.DataFrame()
    
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
#     print('Location:', f)
#     print('File Name:', f.split("\\")[-1])
      
#     # print the content
#     print('Content:')
#     display(df)
#     print()
    master_df = pd.concat([master_df, df])

In [3]:
master_df

,Unnamed: 0,timestamp,image_gt,image_pred,goes16_pred,goes17_pred,final_pred,type
0,222987,2020-07-05 20:27:00+00:00,0.0,0.0,0.0,0.0,0.0,valid
1,222988,2020-07-05 20:28:00+00:00,0.0,0.0,0.0,0.0,0.0,valid
2,222989,2020-07-05 20:29:00+00:00,0.0,0.0,0.0,0.0,0.0,valid
3,222990,2020-07-05 20:30:00+00:00,0.0,0.0,0.0,0.0,0.0,valid
4,222991,2020-07-05 20:31:00+00:00,0.0,0.0,0.0,0.0,0.0,valid
...,...,...,...,...,...,...,...,...
139,459251,2020-12-16 22:11:00+00:00,1.0,1.0,0.0,0.0,0.0,valid
140,459252,2020-12-16 22:12:00+00:00,1.0,1.0,0.0,0.0,0.0,valid
141,459253,2020-12-16 22:13:00+00:00,1.0,1.0,0.0,0.0,0.0,valid
142,459254,2020-12-16 22:14:00+00:00,1.0,1.0,0.0,0.0,0.0,valid


In [4]:
master_copy_df = master_df[master_df["type"] == "test"].copy()
# print(master_df.shape)
# master_df = master_df[(master_df["timestamp"] >= "2021-02-01 00:00:00+00:00") | (master_df["timestamp"] < "2021-01-01 00:00:00+00:00")]
print(master_copy_df.shape)
master_copy_df.sort_values(by="timestamp").to_csv("master_df.csv")

(2733, 8)


In [5]:
master_copy_df.shape

(2733, 8)

In [6]:
image_labels = master_copy_df[~master_copy_df["image_gt"].isna()]["image_gt"]
smokeynet_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["image_pred"]
goes16_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["goes16_pred"]
goes17_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["goes17_pred"]
ensemble_preds = master_copy_df[~master_copy_df["image_gt"].isna()]["final_pred"]

smokeynet_baseline_score = accuracy_score(image_labels, smokeynet_preds)
goes16_baseline_score = accuracy_score(image_labels, goes16_preds)
goes17_baseline_score = accuracy_score(image_labels, goes17_preds)
ensemble_score = accuracy_score(image_labels, ensemble_preds)

print("Baseline score SmokeyNet:", smokeynet_baseline_score)
print("Baseline score GOES-16:", goes16_baseline_score)
print("Baseline score GOES-17:", goes17_baseline_score)
print("Ensemble score:", ensemble_score)

Baseline score SmokeyNet: 0.7907061836809367
Baseline score GOES-16: 0.5181119648737651
Baseline score GOES-17: 0.5781192828393706
Ensemble score: 0.5722649103549213


In [7]:
#smokeynet baseline
precision_recall_fscore_support(image_labels, smokeynet_preds, average = 'binary')

(0.8504201680672269, 0.7197724039829303, 0.7796610169491526, None)

In [8]:
#ensemble 
precision_recall_fscore_support(image_labels, ensemble_preds, average = 'binary')

(1.0, 0.16856330014224752, 0.28849665246500306, None)

In [9]:
#goes16
precision_recall_fscore_support(image_labels, goes16_preds, average = 'binary')

(1.0, 0.06330014224751067, 0.11906354515050169, None)

In [10]:
#goes17
precision_recall_fscore_support(image_labels, goes17_preds, average = 'binary')

(0.9922178988326849, 0.1813655761024182, 0.30667468430547207, None)

In [22]:
master_copy_df["weighted_avg"]  = ((0.7907061836809367 * smokeynet_preds) + (0.5181119648737651 * goes16_preds) + (0.5781192828393706 * goes17_preds)) / (0.7907061836809367 + 0.5181119648737651 + 0.5781192828393706)

#ideally we would need > ~0.275  and <= ~0.306 for the threshold
#we can listen to either SmokeyNet and GOES17
master_copy_df["thresh3"] = master_copy_df["weighted_avg"] >= 0.3
master_copy_df["thresh3"] = master_copy_df["thresh3"].astype(int)

master_copy_df["thresh4"] = master_copy_df["weighted_avg"] >= 0.4
master_copy_df["thresh4"] = master_copy_df["thresh4"].astype(int)

master_copy_df["thresh5"] = master_copy_df["weighted_avg"] >= 0.5
master_copy_df["thresh5"] = master_copy_df["thresh5"].astype(int)

master_copy_df["thresh6"] = master_copy_df["weighted_avg"] >= 0.6
master_copy_df["thresh6"] = master_copy_df["thresh6"].astype(int)

master_copy_df["thresh7"] = master_copy_df["weighted_avg"] >= 0.7
master_copy_df["thresh7"] = master_copy_df["thresh7"].astype(int)

master_copy_df


,Unnamed: 0,timestamp,image_gt,image_pred,goes16_pred,goes17_pred,final_pred,type,weighted_avg,thresh5,thresh3,thresh4,thresh6,thresh7
80,348182,2020-09-30 19:02:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0
81,348183,2020-09-30 19:03:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0
82,348184,2020-09-30 19:04:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0
83,348185,2020-09-30 19:05:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0
84,348186,2020-09-30 19:06:00+00:00,0.0,0.0,0.0,0.0,0.0,test,0.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,531106,2021-02-04 19:46:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,0,1,1,0,0
76,531107,2021-02-04 19:47:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,0,1,1,0,0
77,531108,2021-02-04 19:48:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,0,1,1,0,0
78,531109,2021-02-04 19:49:00+00:00,1.0,1.0,0.0,0.0,0.0,test,0.419042,0,1,1,0,0


In [28]:
#ideally we would need 0.306 for the threshold
accuracy_score(image_labels, master_copy_df["thresh3"])

0.8035126234906695

In [23]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh3"], average = 'binary')

(0.853539462978031, 0.7460881934566145, 0.7962049335863377, None)

In [29]:
accuracy_score(image_labels, master_copy_df["thresh4"])

0.7939992682034395

In [24]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh4"], average = 'binary')

(0.8515429524603837, 0.7261735419630156, 0.7838771593090211, None)

In [30]:
accuracy_score(image_labels, master_copy_df["thresh5"])

0.5722649103549213

In [25]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh5"], average = 'binary')

(1.0, 0.16856330014224752, 0.28849665246500306, None)

In [31]:
accuracy_score(image_labels, master_copy_df["thresh6"])

0.5689718258324186

In [26]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh6"], average = 'binary')

(1.0, 0.16216216216216217, 0.27906976744186046, None)

In [32]:
accuracy_score(image_labels, master_copy_df["thresh7"])

0.5653128430296378

In [27]:
precision_recall_fscore_support(image_labels, master_copy_df["thresh7"], average = 'binary')

(1.0, 0.155049786628734, 0.2684729064039409, None)

In [12]:
precision_recall_fscore_support(image_labels, smokeynet_preds)

(array([0.74465327, 0.85042017]),
 array([0.86586285, 0.7197724 ]),
 array([0.80069686, 0.77966102]),
 array([1327, 1406]))

In [13]:
precision_recall_fscore_support(image_labels, ensemble_preds)

(array([0.53165064, 1.        ]),
 array([1.       , 0.1685633]),
 array([0.6942192 , 0.28849665]),
 array([1327, 1406]))